# Final exam
## Tran Trong Hieu_24MSE23185

In [64]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.measure import label, regionprops
from scipy.ndimage import binary_fill_holes
from scipy.signal import wiener
from skimage.feature import peak_local_max
from skimage.transform import hough_line, hough_line_peaks

def show_image(title, image, cmap = None):
    plt.figure()
    plt.title(title)
    plt.axis('off')
    if cmap:
        plt.imshow(image, cmap = cmap)
    else:
        plt.imshow(image)
    plt.show()

1. Object Counting 

Given a color JPG image named ‘Ex1.jpg’.

(a) Read and display the original image.

(b) Binarize the original image using Otsu method and display the binary image.

(c) Fill small holes in the binary image and display the filled image.

(d) Perform the erosion on the binary image using a suitable structuring element and size.

(e) Apply region labeling on the eroded image to count number of stones in the original image. Print 
the result on the screen.

In [ ]:
# (a) - Read and display the original image
image_path = 'Ex1.jpg'
original_image = cv2.imread(image_path)
original_image_rgb = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB for display
show_image('Original Image', original_image_rgb)

# (b) - Binarize the image using Otsu's method
gray_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2GRAY)
_, binary_image = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
show_image('Binary Image', binary_image, cmap = 'gray')

# (c) - Fill small holes in the binary image
filled_image = binary_fill_holes(binary_image // 255).astype(np.uint8) * 255
show_image('Filled Image', filled_image, cmap = 'gray')

# (d) - Perform erosion on the binary image
se = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (4, 5))
eroded_image = cv2.erode(filled_image, se)
show_image('Eroded Image', eroded_image, cmap = 'gray')

# (e) - Apply region labeling to count objects
labeled_image, num_objects = label(eroded_image, connectivity = 2, return_num = True)

# Create a color label image
colored_labels = np.zeros((*labeled_image.shape, 3), dtype = np.uint8)
for region in regionprops(labeled_image):
    for coord in region.coords:
        colored_labels[coord[0], coord[1]] = np.random.randint(0, 255, size = 3)

show_image('Labeled Image', colored_labels)

# Print the number of objects
print(f'Number of objects: {num_objects}')

2. Denoise and Deblur a Noisy Blurry Grayscale Image 

Given a noisy and blurry grayscale PNG image named ‘Ex2.jpg’.

(a) Read and display the original image.

(b) Method 1: Apply a denoise filter (5x5 averaging kernel filter) then a sharpening filter (h = [0, -1, 0; -1, 6, -1; 0, -1, 0] / 2) to the original image. Display the restored image.

(c) Method 2: Apply the Wiener filter to denoise and deblur the original image. Assume the Gaussian noise variance of the original image is σ=0.04. The estimated noise is calculated by the noise variance divided by the variance of the original image. The sharpening filter using the same with question b. Display the restored image.

(d) Perform the root mean square (rms) calculation in two methods between the restored image and the original image

In [ ]:
# Root Mean Square Error function
def rms_error(original, restored):
    return np.sqrt(np.mean((original - restored) ** 2))

# (a) Read and display the original image
image_path = 'Ex2.jpg'
original_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Read as grayscale
show_image("Original Image", original_image, cmap = 'gray')

# (b) Method 1: Apply denoise filter and sharpening filter
# Denoise using a 5x5 averaging kernel
kernel = np.ones((5, 5), np.float32) / 25
denoised_image = cv2.filter2D(original_image, -1, kernel)

# Sharpen using the given sharpening filter
sharpening_filter = np.array([[0, -1, 0],
                              [-1, 6, -1],
                              [0, -1, 0]]) / 2.0
sharpened_image = cv2.filter2D(denoised_image, -1, sharpening_filter, borderType = cv2.BORDER_REFLECT)
show_image("Restored Image (Method 1)", sharpened_image, cmap = 'gray')

# (c) Method 2: Wiener filter and sharpening
# Calculate estimated noise variance
noise_variance = 0.04
image_variance = np.var(original_image)
estimated_noise = noise_variance / image_variance

# Apply Wiener filter
wiener_image = wiener(original_image, mysize = (5, 5), noise = estimated_noise)

# Sharpen the Wiener-filtered image
sharpened_wiener_image = cv2.filter2D(wiener_image, -1, sharpening_filter, borderType = cv2.BORDER_REFLECT)
show_image("Restored Image (Method 2)", sharpened_wiener_image, cmap = 'gray')

# (d) Perform RMS calculation
rms_method1 = rms_error(original_image, sharpened_image)
rms_method2 = rms_error(original_image, sharpened_wiener_image)

print(f"RMS Error (Method 1): {rms_method1}")
print(f"RMS Error (Method 2): {rms_method2}")


3. Edge Detection and Hough Transform 

Given a color JPG image named ‘Ex3.jpg’.

(a) Read and display the original image in color and grayscale format.

(b) Apply Roberts filter in two directions (horizontal [1 0; 0 -1] and vertical [0 1; -1 0]). Display the logarithm of sum of edge magnitude response with a threhold value = 5000. Then, display the binary image based on this threshold.

(c) Apply Canny Edge Detector using a suitable standard deviation σ and a suitable threshold value.

(d) Apply Hough Transform using a ratio = 0.2 of max peak and NhoodSize = [49 49]

In [ ]:
# (a) Read and display the original image in color and grayscale format
image_path = 'Ex3.jpg'
original_image = cv2.imread(image_path)  # Read as a color image
original_image_rgb = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB for display
gray_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2GRAY)  # Convert to grayscale

show_image("Original Image (Color)", original_image_rgb)
show_image("Original Image (Grayscale)", gray_image, cmap = 'gray')

# (b) Apply Roberts filter in two directions
# Define Roberts filters
roberts_horizontal = np.array([[1, 0], [0, -1]])
roberts_vertical = np.array([[0, 1], [-1, 0]])

# Apply filters
horizontal_edges = cv2.filter2D(gray_image.astype(float), -1, roberts_horizontal)
vertical_edges = cv2.filter2D(gray_image.astype(float), -1, roberts_vertical)

# Compute edge magnitude
edge_magnitude = horizontal_edges ** 2 + vertical_edges ** 2

# Apply logarithmic transformation
log_edge_magnitude = np.log(1 + edge_magnitude)
log_edge_magnitude = log_edge_magnitude / np.max(log_edge_magnitude)

# Thresholding (threshold = 5000)
threshold_value = 4000
binary_image = (edge_magnitude > threshold_value).astype(np.uint8)

show_image("Log Edge Magnitude (Roberts)", log_edge_magnitude, cmap = 'gray')
show_image("Binary Image (Threshold > 5000)", binary_image, cmap = 'gray')

# (c) Apply Canny Edge Detector
# Set suitable parameters (adjust if needed)
sigma = 1.0  # Standard deviation for Gaussian blur
low_threshold = 50  # Lower threshold for edge detection
high_threshold = 150  # Upper threshold for edge detection

# Apply Canny Edge Detector
canny_edges = cv2.Canny(gray_image, low_threshold, high_threshold)
show_image("Canny Edge Detector", canny_edges, cmap = 'gray')

# (d) Apply Hough Transform
# Hough Transform
h, theta, d = hough_line(canny_edges)

# Find peaks in Hough transform
_, angles, dists = hough_line_peaks(h, theta, d, threshold = 0.8 * np.max(h))

# Apply Hough Line Transform
# You can adjust the parameters to filter out the unwanted lines
lines = cv2.HoughLinesP(canny_edges, 1, np.pi / 90, threshold = 100, minLineLength = 50, maxLineGap = 2)

# Create a copy of the original image to draw the lines on
# Create a copy of the original image to draw the lines on
img_color = original_image_rgb.copy()

# Draw the detected lines on the image
if lines is not None:
    for line in lines:
        x1, y1, x2, y2 = line[0]
        # Draw the line on the image with green color and line width of 2
        cv2.line(img_color, (x1, y1), (x2, y2), (0, 255, 0), 2)

show_image("Hough Transform Lines", img_color)


4. Key point Detection 

Given a color JPG image named ‘Ex4.jpg’.

(a) Read and display the original image in color and grayscale format.
 
(b) Use LoG to highlight the 100 strongest key points in the original image. Choose sigma value = 2*sqrt(2). 

(c) Use DoH to highlight the 100 strongest key points in the original image. Choose sigma value = 3. 

(d) Use FAST to highlight the 100 strongest key points in the original image. Choose the threshold value = 0.2. 

All cases used Prewitt operator if needed. Other parameters are optional.

In [ ]:
# (a) Read and display the original image in color and grayscale format
image_path = "Ex4.jpg"
original_image = cv2.imread(image_path)
original_image_rgb = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
gray_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2GRAY)  # Convert to grayscale

show_image("Original Image (Color)", original_image_rgb)
show_image("Original Image (Grayscale)", gray_image, cmap = "gray")

# (b) Laplacian of Gaussian (LoG) keypoint detection
sigma_log = 2 * np.sqrt(2)
log_response = cv2.GaussianBlur(gray_image, (0, 0), sigmaX = sigma_log, sigmaY = sigma_log, borderType = cv2.BORDER_DEFAULT)
log_response = cv2.Laplacian(log_response, cv2.CV_64F)
thresholded_log = log_response.copy()
thresholded_log[thresholded_log < 0.05 * np.max(log_response)] = 0

local_maxima = peak_local_max(log_response, min_distance = 5, num_peaks = 100)
img_keypoints_log = original_image.copy()
for point in local_maxima:
    cv2.drawMarker(img_keypoints_log, tuple(point[::-1]), (0, 0, 255), markerType = cv2.MARKER_CROSS, markerSize = 10)

show_image("LoG Response", log_response, cmap = "gray")
show_image("Thresholded LoG Response", thresholded_log, cmap = "gray")
show_image("Local Extrema (Dilated)", thresholded_log > 0, cmap = "gray")
show_image("Keypoints (LoG)", cv2.cvtColor(img_keypoints_log, cv2.COLOR_BGR2RGB))

# (c) Determinant of Hessian (DoH) keypoint detection
sigma_doh = 3
gaussian = cv2.GaussianBlur(gray_image, (0, 0), sigmaX = sigma_doh, sigmaY = sigma_doh, borderType = cv2.BORDER_DEFAULT)
dxx = cv2.Sobel(gaussian, cv2.CV_64F, 2, 0, ksize = 5)
dyy = cv2.Sobel(gaussian, cv2.CV_64F, 0, 2, ksize = 5)
dxy = cv2.Sobel(gaussian, cv2.CV_64F, 1, 1, ksize = 5)
doh_response = dxx * dyy - dxy**2

thresholded_doh = doh_response.copy()
thresholded_doh[thresholded_doh < 0.05 * np.max(doh_response)] = 0

local_maxima_doh = peak_local_max(doh_response, min_distance = 5, num_peaks = 100)
img_keypoints_doh = original_image_rgb.copy()
for point in local_maxima_doh:
    cv2.drawMarker(img_keypoints_doh, tuple(point[::-1]), (255, 0, 0), markerType = cv2.MARKER_CROSS, markerSize = 10)

show_image("DoH Response", doh_response, cmap = "gray")
show_image("Thresholded DoH Response", thresholded_doh, cmap = "gray")
show_image("Local Maxima (Dilated)", thresholded_doh > 0, cmap = "gray")
show_image("Keypoints (DoH)", cv2.cvtColor(img_keypoints_doh, cv2.COLOR_BGR2RGB))

# (d) FAST Keypoint Detection
fast = cv2.FastFeatureDetector_create(threshold = 20, nonmaxSuppression = False)
keypoints = fast.detect(gray_image, None)

img_keypoints_fast = cv2.drawKeypoints(original_image_rgb, keypoints, None, color = (0, 255, 0))

fast.setNonmaxSuppression(True)
keypoints_nonmax = fast.detect(gray_image, None)

img_keypoints_fast_nonmax = cv2.drawKeypoints(original_image_rgb, keypoints_nonmax, None, color = (0, 0, 255))

show_image("Fast Corners", cv2.cvtColor(img_keypoints_fast, cv2.COLOR_BGR2RGB))
show_image("Fast Corners, Non-max Suppressed", cv2.cvtColor(img_keypoints_fast_nonmax, cv2.COLOR_BGR2RGB))
